In [1]:
# config and imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

DATA_PATH = "Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
TARGET_STATE = "GA"
CUTOFF_COL = "2024-02-29"      # last column to use for features one year before target
TARGET_COL = "2025-02-28"      # target column to predict


In [2]:
# helper functions for trimming, growth, and volatility

def get_zhvi_columns(df):
    return df.columns[df.columns.str.match(r'^\d{4}-\d{2}-\d{2}$')]

def longest_nan_streak(row):
    is_null = row.isnull()
    max_streak = 0
    current_streak = 0
    for val in is_null:
        if val:
            current_streak += 1
            max_streak = max(max_streak, current_streak)
        else:
            current_streak = 0
    return max_streak

def trim_to_first_valid(series):
    first_valid = series.first_valid_index()
    if first_valid:
        return series.loc[first_valid:]
    return pd.Series(dtype='float64')

# calculates percent change from n months ago
def yoy_growth(series, months_back=12):
    if len(series) < months_back + 1 or series.iloc[-months_back - 1] == 0:
        return np.nan
    return 100 * (series.iloc[-1] - series.iloc[-months_back - 1]) / series.iloc[-months_back - 1]

# calculates total percent growth over entire series
def total_growth(series):
    if len(series) < 2 or series.iloc[0] == 0:
        return np.nan
    return 100 * (series.iloc[-1] - series.iloc[0]) / series.iloc[0]

# calculates compound annual growth rate
def cagr(series):
    if len(series) < 2 or series.iloc[0] <= 0:
        return np.nan
    years = len(series) / 12
    return ((series.iloc[-1] / series.iloc[0]) ** (1 / years) - 1) * 100

# calculates average monthly percent change
def avg_monthly_growth(series):
    monthly_pct = series.pct_change()
    return 100 * monthly_pct.mean()

# calculates standard deviation of monthly percent changes
def volatility(series):
    monthly_pct = series.pct_change()
    return 100 * monthly_pct.std()

# calculates yoy growth for each full year with available data
def compute_yoy_by_year(series):
    series = series.copy()
    series.index = pd.to_datetime(series.index)
    yearly_growth = {}

    for year in range(series.index.year.min() + 1, series.index.year.max() + 1):
        try:
            prev = series[series.index.year == year - 1].iloc[-1]
            curr = series[series.index.year == year].iloc[-1]
            if pd.notna(prev) and prev != 0:
                growth = 100 * (curr - prev) / prev
                yearly_growth[year] = growth
        except IndexError:
            continue

    return pd.Series(yearly_growth)


In [3]:
# loads dataset, fills metadata gaps, trims zhvi, interpolates missing values

df = pd.read_csv(DATA_PATH)
df = df[df["State"] == TARGET_STATE].copy()
print(f"loaded {df.shape[0]} ZIPs from {TARGET_STATE}")

zhvi_cols = get_zhvi_columns(df)

# check initial missing metadata counts
meta_cols = ["RegionID", "RegionName", "City", "Metro", "CountyName", "SizeRank", "State"]
missing_meta = df[meta_cols].isnull().sum()
missing_meta = missing_meta[missing_meta > 0]

both_missing = df[df["City"].isnull() & df["Metro"].isnull()].shape[0]
only_city_missing = df[df["City"].isnull() & df["Metro"].notnull()].shape[0]
only_metro_missing = df[df["Metro"].isnull() & df["City"].notnull()].shape[0]

print("\nmissing metadata breakdown:")
print(f"missing both city and metro: {both_missing}")
print(f"missing only city: {only_city_missing}")
print(f"missing only metro: {only_metro_missing}")

# fill metro using city → metro mapping, avoiding ambiguous cities
ambiguous_city_set = {"Boston", "Tifton"}
safe_city_to_metro = (
    df[df["City"].notna() & df["Metro"].notna() & ~df["City"].isin(ambiguous_city_set)]
    .groupby("City")["Metro"]
    .agg(lambda x: x.mode().iloc[0])
    .to_dict()
)

df["Metro"] = df.apply(
    lambda row: safe_city_to_metro.get(row["City"], row["Metro"]) if pd.isna(row["Metro"]) else row["Metro"],
    axis=1
)

# fill remaining metro using county to metro mapping
county_to_metro = (
    df[df["Metro"].notna()][["CountyName", "Metro"]]
    .drop_duplicates()
    .groupby("CountyName")["Metro"]
    .agg(lambda x: x.mode().iloc[0])
    .to_dict()
)

df["Metro"] = df.apply(
    lambda row: county_to_metro.get(row["CountyName"], row["Metro"]) if pd.isna(row["Metro"]) else row["Metro"],
    axis=1
)

# drop rows missing both city and metro, then drop rows missing either
df = df[~(df["City"].isnull() & df["Metro"].isnull())].copy()
before_drop = df.shape[0]
df = df[df["City"].notna() & df["Metro"].notna()].copy()
after_drop = df.shape[0]

print(f"\nmetadata cleanup:")
print(f"rows before drop: {before_drop}")
print(f"rows after drop: {after_drop}")
print(f"rows removed: {before_drop - after_drop}")

# trim zhvi series to first valid value and up to cutoff
cutoff_cols = [col for col in zhvi_cols if col <= CUTOFF_COL]
trimmed_series_list, trimmed_lengths, missing_counts = [], [], []

for _, row in df.iterrows():
    trimmed = trim_to_first_valid(row[cutoff_cols]).astype("float64")
    trimmed_series_list.append(trimmed)
    trimmed_lengths.append(len(trimmed))
    missing_counts.append(trimmed.isnull().sum())

df["TrimmedZHVI"] = trimmed_series_list
df["TrimmedLength"] = trimmed_lengths
df["TrimmedMissing"] = missing_counts

print(f"\ntrimmed zhvi series for {len(df)} ZIPs")

# interpolate missing values within each trimmed series
interpolated_series, interpolated_missing_counts = [], []
for s in df["TrimmedZHVI"]:
    interpolated = s.interpolate(method="linear", limit_direction="both")
    interpolated_series.append(interpolated)
    interpolated_missing_counts.append(interpolated.isna().sum())

df["InterpolatedZHVI"] = interpolated_series
df["InterpolatedMissing"] = interpolated_missing_counts

before_interp = df.shape[0]
df = df[df["InterpolatedMissing"] == 0].copy()
after_interp = df.shape[0]

print(f"\ninterpolation result:")
print(f"clean rows: {after_interp}")
print(f"removed rows: {before_interp - after_interp}")

# define target as the most recent zhvi value in the full series
df[TARGET_COL] = df["InterpolatedZHVI"].apply(lambda s: s.iloc[-1])


loaded 665 ZIPs from GA

missing metadata breakdown:
missing both city and metro: 11
missing only city: 25
missing only metro: 111

metadata cleanup:
rows before drop: 658
rows after drop: 558
rows removed: 100

trimmed zhvi series for 558 ZIPs

interpolation result:
clean rows: 558
removed rows: 0


In [4]:
# computes engineered features using interpolated zhvi

# final zhvi is the last value in the trimmed series
df["FinalZHVI"] = df["InterpolatedZHVI"].apply(lambda s: s.iloc[-1])

# z-score across all final values
df["FinalZScore"] = zscore(df["FinalZHVI"])

# assign value tier using asymmetric z-score bins
def zhvi_asymmetric_tier(z):
    if z >= 2.5:
        return "very high"
    elif z >= 1.5:
        return "high"
    elif z >= -0.5:
        return "mid"
    elif z >= -1.0:
        return "low"
    else:
        return "very low"

df["ZHVI_Tier"] = df["FinalZScore"].apply(zhvi_asymmetric_tier)

# compute yoy growth per year from the interpolated series
yoy_df = df["InterpolatedZHVI"].apply(compute_yoy_by_year)
yoy_df = yoy_df.add_prefix("YoY_")
df = pd.concat([df, yoy_df], axis=1)

# compute aggregate features from the yearly growth
df["AvgYoYGrowth"] = yoy_df.mean(axis=1)
df["MedianYoYGrowth"] = yoy_df.median(axis=1)
df["YoYGrowthVolatility"] = yoy_df.std(axis=1)
df["NegativeGrowthYears"] = (yoy_df < 0).sum(axis=1)

# compute full-period features using the interpolated series
df["Total_Growth"] = df["InterpolatedZHVI"].apply(total_growth)
df["CAGR"] = df["InterpolatedZHVI"].apply(cagr)
df["AvgMonthlyGrowth"] = df["InterpolatedZHVI"].apply(avg_monthly_growth)
df["Volatility"] = df["InterpolatedZHVI"].apply(volatility)

# show distribution of value tiers
print("zhvi tier distribution:")
print(df["ZHVI_Tier"].value_counts().sort_index())



zhvi tier distribution:
ZHVI_Tier
high          22
low          128
mid          323
very high     18
very low      67
Name: count, dtype: int64


In [5]:
# prepares final export dataset and writes to csv

# assign new state-specific rank based on original SizeRank values
df["StateSizeRank"] = df["SizeRank"].rank(method="min").astype(int)

# drop RegionID and original SizeRank
df.drop(columns=["RegionID", "SizeRank"], inplace=True)

# rename RegionName to ZIP for clarity
df.rename(columns={"RegionName": "ZIP"}, inplace=True)

# columns to drop 
drop_cols = [
    col for col in df.columns
    if (col in [
        "State", "StateName", "RegionType", "TrimmedZHVI", "InterpolatedZHVI",
        "TrimmedLength", "TrimmedMissing", "InterpolatedMissing"
    ] or (col in zhvi_cols and col != TARGET_COL) or col.startswith("YoY_")) 
]

# create final cleaned version for modeling
export_df = df.drop(columns=drop_cols)

# reorder columns 
meta_cols = ["ZIP", "City", "Metro", "CountyName", "StateSizeRank"]
target_col = TARGET_COL
feature_cols = [col for col in export_df.columns if col not in meta_cols + [target_col]]
ordered_cols = meta_cols + feature_cols + [target_col]
export_df = export_df[ordered_cols]

# shuffle rows
export_df = export_df.sample(frac=1, random_state=42).reset_index(drop=True)

# export to csv
print("final dataset preview:")
display(export_df.head(0))

export_df.to_csv("final_processed_dataset.csv", index=False)
print(f"exported cleaned dataset with {export_df.shape[0]} rows and {export_df.shape[1]} columns")


final dataset preview:


,ZIP,City,Metro,CountyName,StateSizeRank,FinalZHVI,FinalZScore,ZHVI_Tier,AvgYoYGrowth,MedianYoYGrowth,YoYGrowthVolatility,NegativeGrowthYears,Total_Growth,CAGR,AvgMonthlyGrowth,Volatility,2025-02-28


exported cleaned dataset with 558 rows and 17 columns
